In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import joblib
from scipy.sparse import hstack

In [9]:
#VAMOS A VER QUE HACE Pycaret
#Al querer predecir un número vamos a usar regression en lugar de clasiification que es para predecir categorías

from pycaret.regression import *
import pandas as pd

movies = pd.read_csv('movies_Final.csv')

# Inicializar el entorno de PyCaret

exp_reg101 = setup(data=movies, target='revenue', session_id=123)

# Comparar varios modelos de regresión
best = compare_models()

#El resultado mandando el ds es que el mejor modelo es K Neighbors Regressor
#voy a lanzarlo otra vez pero con los campos vectorizados

# Convertir la columna release_date a datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

# Rellenar los valores nulos en las columnas importantes
movies['actors'] = movies['actors'].fillna('')
movies['directors_name'] = movies['directors_name'].fillna('')
movies['production_companies'] = movies['production_companies'].fillna('')
movies['genres'] = movies['genres'].fillna('')


,Description,Value
0,Session id,123
1,Target,revenue
2,Target type,Regression
3,Original data shape,"(14160, 33)"
4,Transformed data shape,"(14160, 37)"
5,Transformed train set shape,"(9912, 37)"
6,Transformed test set shape,"(4248, 37)"
7,Numeric features,11
8,Categorical features,19
9,Rows with missing values,36.6%


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,44368907.4970,12705191061732458.0000,111942605.1038,0.2318,8.7135,27735.8421,0.2750
par,Passive Aggressive Regressor,59673668.9345,14551586715801180.0000,119938220.7797,0.1156,10.2592,56082.7149,0.2490
lightgbm,Light Gradient Boosting Machine,61664615.4491,16267570327870668.0000,126817454.1054,0.0121,10.3471,63565.6654,0.5030
et,Extra Trees Regressor,61708522.7135,16372465419746492.0000,127266126.3351,0.0046,10.3466,65182.8738,1.1830
gbr,Gradient Boosting Regressor,61813400.6864,16403314596648648.0000,127387028.9763,0.0026,10.3493,65117.5138,1.2740
rf,Random Forest Regressor,61672916.0577,16423022751215484.0000,127458190.5020,0.0016,10.3440,64353.8148,2.4940
xgboost,Extreme Gradient Boosting,61964958.8437,16430565689769610.0000,127485293.1375,0.0012,10.3531,66349.1758,0.3840
dt,Decision Tree Regressor,61804040.7947,16449826706415558.0000,127558933.5000,0.0001,10.3481,63856.4937,0.2770
en,Elastic Net,61900595.3817,16451496872721888.0000,127568585.6805,-0.0001,10.3506,66307.4057,0.3820
lasso,Lasso Regression,61897416.5412,16451381624014282.0000,127568214.3118,-0.0001,10.3503,66296.7165,0.5630


In [1]:
import pycaret
pycaret.__version__

'3.3.1'

In [9]:
import os

print(os.getcwd())

c:\Users\mfrod\Downloads


Mas pruebas, ahora voy a sumar las peliculas futuras al proceso, primero con RandomForestRegressor

In [10]:
#######Este es el modelo que más me gusta: RandomForestRegressor

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import hstack
import joblib

# Cargar los datos
movies = pd.read_csv('movies_Final.csv')
futuras = pd.read_csv('dataset_futuras.csv')

# Convertir la columna release_date a datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
futuras['release_date'] = pd.to_datetime(futuras['release_date'], errors='coerce')

# Rellenar los valores nulos en las columnas importantes
movies['actors'] = movies['actors'].fillna('')
movies['directors_name'] = movies['directors_name'].fillna('')
movies['production_companies'] = movies['production_companies'].fillna('')
movies['genres'] = movies['genres'].fillna('')
futuras['actors'] = futuras['actors'].fillna('')
futuras['directors_name'] = futuras['directors_name'].fillna('')
futuras['production_companies'] = futuras['production_companies'].fillna('')
futuras['genres'] = futuras['genres'].fillna('')

# Vectorizar las características textuales
all_actors = pd.concat([movies['actors'], futuras['actors']])
all_directors = pd.concat([movies['directors_name'], futuras['directors_name']])
all_companies = pd.concat([movies['production_companies'], futuras['production_companies']])
all_genres = pd.concat([movies['genres'], futuras['genres']])

# Entrenar vectorizadores
vectorizer_actors = CountVectorizer()
vectorizer_actors.fit(all_actors)

vectorizer_directors = CountVectorizer()
vectorizer_directors.fit(all_directors)

vectorizer_companies = CountVectorizer()
vectorizer_companies.fit(all_companies)

vectorizer_genres = CountVectorizer()
vectorizer_genres.fit(all_genres)

# Transformar datos de películas estrenadas
actors_matrix = vectorizer_actors.transform(movies['actors'])
directors_matrix = vectorizer_directors.transform(movies['directors_name'])
companies_matrix = vectorizer_companies.transform(movies['production_companies'])
genres_matrix = vectorizer_genres.transform(movies['genres'])

# Transformar datos de futuras películas
actors_matrix_futuras = vectorizer_actors.transform(futuras['actors'])
directors_matrix_futuras = vectorizer_directors.transform(futuras['directors_name'])
companies_matrix_futuras = vectorizer_companies.transform(futuras['production_companies'])
genres_matrix_futuras = vectorizer_genres.transform(futuras['genres'])

# Concatenar todas las características
X = hstack([actors_matrix, directors_matrix, companies_matrix, genres_matrix])

# Estandarizar características numéricas
scaler = StandardScaler(with_mean=False)  
X = scaler.fit_transform(X)

# Guardar los vectorizadores y el escalador
joblib.dump(vectorizer_actors, 'vectorizer_actors.pkl')
joblib.dump(vectorizer_directors, 'vectorizer_directors.pkl')
joblib.dump(vectorizer_companies, 'vectorizer_companies.pkl')
joblib.dump(vectorizer_genres, 'vectorizer_genres.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Variable objetivo
y = np.log1p(movies['revenue'])

# dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Guardar el modelo
joblib.dump(model, 'revenue_predictor.pkl')

# Predecir y evaluar el modelo
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # Transformación inversa del logaritmo
y_test_original = np.expm1(y_test)  # Transformación inversa del logaritmo

mse = mean_squared_error(y_test_original, y_pred)
print(f'Error cuadrático medio: {mse}')

# Seleccionar las 10 películas con el revenue más alto del conjunto de prueba
top_10_indices = y_test_original.nlargest(10).index
top_10_movies = movies.loc[top_10_indices]
top_10_real_revenue = y_test_original.loc[top_10_indices]
top_10_predicted_revenue = pd.Series(y_pred, index=y_test.index).loc[top_10_indices]

# Crear un DataFrame con los resultados y formatear los valores
result_df = pd.DataFrame({
    'title': top_10_movies['title'].values,
    'real_revenue': top_10_real_revenue.apply(lambda x: f"${x:,.0f}"),
    'predicted_revenue': top_10_predicted_revenue.apply(lambda x: f"${x:,.0f}")
})

print(result_df)

# Ahora aplicamos el modelo sobre las futuras películas
# Transformar y escalar las características usando los objetos cargados
X_futuras = hstack([actors_matrix_futuras, directors_matrix_futuras, companies_matrix_futuras, genres_matrix_futuras])
X_futuras = scaler.transform(X_futuras)

# Predecir con el modelo entrenado
y_futuras_pred_log = model.predict(X_futuras)
y_futuras_pred = np.expm1(y_futuras_pred_log)

futuras['predicted_revenue'] = y_futuras_pred

# Seleccionar las 10 películas con la predicción de revenue más alto
top_10_futuras = futuras.nlargest(10, 'predicted_revenue').copy()

# Formatear los valores de predicción de revenue
top_10_futuras['predicted_revenue'] = top_10_futuras['predicted_revenue'].apply(lambda x: f"${x:,.0f}")

print(top_10_futuras[['title', 'predicted_revenue']])


Error cuadrático medio: 1.0305985612586204e+16
                                           title    real_revenue  \
15                             Avengers: Endgame  $2,800,000,000   
248               Jurassic World: Fallen Kingdom  $1,310,466,296   
99                                        Frozen  $1,274,219,009   
113                               Captain Marvel  $1,131,416,446   
428              Transformers: Age of Extinction  $1,104,054,072   
330             Star Wars: The Rise of Skywalker  $1,074,144,248   
149  Pirates of the Caribbean: On Stranger Tides  $1,045,713,802   
538                              Despicable Me 3  $1,031,552,585   
101                                     Zootopia  $1,023,784,195   
59        Harry Potter and the Half-Blood Prince    $933,959,197   

    predicted_revenue  
15     $1,423,398,897  
248      $421,396,816  
99       $397,792,559  
113      $598,469,067  
428      $308,350,356  
330      $938,612,287  
149      $340,274,247  
538      $22

In [4]:
futuras.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,año,era,decade,title_es,runtimeMinutes,averageRating,numVotes,actors,directors_name,produced_in_USA_or_UK
0,975056,Cabrini,0.0,0,In Production,2024-03-08,0,0,False,/os4UQLyj0GkqarDwPY73qSzgNhY.jpg,...,2024,Era del Cine Contemporáneo y la Globalización,2020.0,Cabrini,142.0,7.7,4176.0,"Giancarlo Giannini, John Lithgow, David Morse,...",Alejandro Monteverde,True
1,974036,Ordinary Angels,0.0,0,Post Production,2024-02-23,0,116,False,/wrzHQXtcnunPjrbfwlPLIr2TRRC.jpg,...,2024,Era del Cine Contemporáneo y la Globalización,2020.0,Ordinary Angels,118.0,7.4,4894.0,"Nancy Travis, Hilary Swank, Tamala Jones, Drew...",Jon Gunn,True
2,986056,Thunderbolts,0.0,0,Planned,2024-12-18,0,0,False,/vzVojqW0zF9QrXVGuj8PwhxWACD.jpg,...,2024,Era del Cine Contemporáneo y la Globalización,2020.0,Thunderbolts,0.0,NaN,NaN,"Julia Louis-Dreyfus, Rachel Weisz, David Harbo...",Jake Schreier,True
3,1262868,Woodbridge,0.0,0,Post Production,2024-04-01,0,85,False,NaN,...,2024,Era del Cine Contemporáneo y la Globalización,2020.0,Woodbridge,85.0,NaN,NaN,"Sophi Knight, Carrington Bornstein, Julia Mast...",Stephen Meier,True
4,1022787,Elio,0.0,0,In Production,2024-02-28,0,90,False,/oLN4E0pX0z0wLnlfyn8vwnuGBYF.jpg,...,2024,Era del Cine Contemporáneo y la Globalización,2020.0,Elío,0.0,NaN,NaN,"Brad Garrett, America Ferrera, Yonas Kibreab, ...",Adrian Molina,True


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14160 entries, 0 to 14159
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14160 non-null  int64  
 1   title                  14160 non-null  object 
 2   vote_average           14160 non-null  float64
 3   vote_count             14160 non-null  int64  
 4   status                 14160 non-null  object 
 5   release_date           14160 non-null  object 
 6   revenue                14160 non-null  int64  
 7   runtime                14160 non-null  int64  
 8   adult                  14160 non-null  bool   
 9   budget                 14160 non-null  int64  
 10  imdb_id                14160 non-null  object 
 11  original_language      14160 non-null  object 
 12  original_title         14160 non-null  object 
 13  overview               14160 non-null  object 
 14  popularity             14160 non-null  float64
 15  po

In [3]:
import pandas as pd
import numpy as np
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Descargar los recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Cargar los DataFrames
movies = pd.read_csv('movies_Final.csv')
futuras = pd.read_csv('dataset_futuras.csv')

# Función para preprocesar texto
def preprocess_text(text):
    if pd.isnull(text):
        return ''
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Combinar las columnas de texto en una sola columna
def combine_text_columns(df):
    df['combined_text'] = df['actors'] + ' ' + df['directors_name'] + ' ' + df['keywords'] + ' ' + df['production_companies'] + ' ' + df['genres']
    df['combined_text'] = df['combined_text'].apply(preprocess_text)
    return df

movies = combine_text_columns(movies)
futuras = combine_text_columns(futuras)

# Rellenar valores nulos
movies.fillna('', inplace=True)
futuras.fillna('', inplace=True)

# Inicializar el vectorizador TF-IDF
tfidf = TfidfVectorizer(max_features=5000)

# Ajustar y transformar los datos de entrenamiento
X = tfidf.fit_transform(movies['combined_text'])

# Transformar los datos futuros
X_futuras = tfidf.transform(futuras['combined_text'])

# Variable objetivo (y) y características (X)
y = movies['revenue']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluar el modelo
print(f"Score del modelo en el conjunto de prueba: {model.score(X_test, y_test)}")

# Hacer predicciones
predictions = model.predict(X_futuras)

# Añadir las predicciones al DataFrame de futuras películas
futuras['predicted_revenue'] = predictions




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mpata_lrc8dz0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mpata_lrc8dz0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Score del modelo en el conjunto de prueba: -0.09476202456925598


In [ ]:
##Despues de ver los resultados nos quedamos con este modelo

In [4]:
# Ordenar por los ingresos previstos y seleccionar las 15 principales
top_15_movies_futuras = futuras.nlargest(15, 'predicted_revenue')

# Ajustar la visualización para evitar la notación científica
pd.set_option('display.float_format', '{:.2f}'.format)

# Mostrar las 15 películas futuras con mayores ingresos estimados
print("Top 15 Películas Futuras con Mayores Ingresos Estimados:")
print(top_15_movies_futuras[['title', 'predicted_revenue']])

# Ordenar las películas ya estrenadas por ingresos reales y seleccionar las 15 principales
top_15_movies_actuales = movies.nlargest(15, 'revenue')

# Mostrar las 15 películas ya estrenadas con mayores ingresos reales
print("\nTop 15 Películas Estrenadas con Mayores Ingresos Reales:")
print(top_15_movies_actuales[['title', 'revenue']])
#vamos a exportar top_15_movies_futuras a un csv
top_15_movies_futuras.to_csv('top_15_movies_futuras.csv', index=False)


Top 15 Películas Futuras con Mayores Ingresos Estimados:
                               title  predicted_revenue
51                          Avatar 5      1060246840.50
49                          Avatar 4      1047534551.64
48                          Avatar 3      1034375639.43
68                             Blade      1020470549.75
5                       Inside Out 2       953916505.01
44                   Despicable Me 4       827037169.88
45                        Deadpool 3       790607084.57
2                       Thunderbolts       703165770.45
40                        Snow White       574682570.01
31                           Moana 2       554636509.13
14               Joker: Folie à Deux       476801946.89
61  Captain America: Brave New World       467173245.56
73                          Twisters       406700736.61
76                    Dune: Part Two       400701458.57
23              Sonic the Hedgehog 3       387865713.27

Top 15 Películas Estrenadas con Mayores Ingres

In [8]:
#Si solo quiero usar el modelo entrenado puedo usar esta parte del código:
import pandas as pd
import joblib
from scipy.sparse import hstack

# Cargar los datos
futuras = pd.read_csv('dataset_futuras.csv')

# Convertir la columna release_date a datetime
futuras['release_date'] = pd.to_datetime(futuras['release_date'], errors='coerce')

# Rellenar los valores nulos en las columnas importantes
futuras['actors'] = futuras['actors'].fillna('')
futuras['directors_name'] = futuras['directors_name'].fillna('')
futuras['production_companies'] = futuras['production_companies'].fillna('')
futuras['genres'] = futuras['genres'].fillna('')

# Cargar los vectorizadores y el escalador
vectorizer_actors = joblib.load('vectorizer_actors.pkl')
vectorizer_directors = joblib.load('vectorizer_directors.pkl')
vectorizer_companies = joblib.load('vectorizer_companies.pkl')
vectorizer_genres = joblib.load('vectorizer_genres.pkl')
scaler = joblib.load('scaler.pkl')
model = joblib.load('revenue_predictor.pkl')

# Transformar datos de futuras películas
actors_matrix_futuras = vectorizer_actors.transform(futuras['actors'])
directors_matrix_futuras = vectorizer_directors.transform(futuras['directors_name'])
companies_matrix_futuras = vectorizer_companies.transform(futuras['production_companies'])
genres_matrix_futuras = vectorizer_genres.transform(futuras['genres'])

# Concatenar todas las características y escalar
X_futuras = hstack([actors_matrix_futuras, directors_matrix_futuras, companies_matrix_futuras, genres_matrix_futuras])
X_futuras = scaler.transform(X_futuras)

# Predecir con el modelo cargado
y_futuras_pred_log = model.predict(X_futuras)
y_futuras_pred = np.expm1(y_futuras_pred_log)

futuras['predicted_revenue'] = y_futuras_pred

# Seleccionar las 10 películas con la predicción de revenue más alto
top_10_futuras = futuras.nlargest(10, 'predicted_revenue').copy()

# Formatear los valores de predicción de revenue
top_10_futuras['predicted_revenue'] = top_10_futuras['predicted_revenue'].apply(lambda x: f"${x:,.0f}")

print(top_10_futuras[['title', 'predicted_revenue']])



                                            title predicted_revenue
42  Mission: Impossible - Dead Reckoning Part Two      $445,392,683
4                                            Elio      $314,442,693
31                                        Moana 2      $293,862,403
5                                    Inside Out 2      $264,073,381
73                                       Twisters      $253,591,545
51                                       Avatar 5      $192,230,644
48                                       Avatar 3      $182,836,912
44                                Despicable Me 4      $170,804,341
14                            Joker: Folie à Deux      $163,471,416
49                                       Avatar 4      $160,174,188


Ahora vamos a usar el modelo que nos recomienda Pycaret KNN
Los resultados como títulos de peliculas suenan coerentes pero la variable objetivo queda mucho más alejada de la realidad que el anterior modelo

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from scipy.sparse import hstack
import joblib
import time

# Cargar los datos
movies = pd.read_csv('movies_Final.csv')
futuras = pd.read_csv('dataset_futuras.csv')

# Convertir la columna release_date a datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
futuras['release_date'] = pd.to_datetime(futuras['release_date'], errors='coerce')

# Rellenar los valores nulos en las columnas importantes
movies['actors'] = movies['actors'].fillna('')
movies['directors_name'] = movies['directors_name'].fillna('')
movies['production_companies'] = movies['production_companies'].fillna('')
movies['genres'] = movies['genres'].fillna('')
futuras['actors'] = futuras['actors'].fillna('')
futuras['directors_name'] = futuras['directors_name'].fillna('')
futuras['production_companies'] = futuras['production_companies'].fillna('')
futuras['genres'] = futuras['genres'].fillna('')

# Vectorizar las características textuales
all_actors = pd.concat([movies['actors'], futuras['actors']])
all_directors = pd.concat([movies['directors_name'], futuras['directors_name']])
all_companies = pd.concat([movies['production_companies'], futuras['production_companies']])
all_genres = pd.concat([movies['genres'], futuras['genres']])

# Entrenar vectorizadores
vectorizer_actors = CountVectorizer()
vectorizer_actors.fit(all_actors)

vectorizer_directors = CountVectorizer()
vectorizer_directors.fit(all_directors)

vectorizer_companies = CountVectorizer()
vectorizer_companies.fit(all_companies)

vectorizer_genres = CountVectorizer()
vectorizer_genres.fit(all_genres)

# Transformar datos de películas estrenadas
actors_matrix = vectorizer_actors.transform(movies['actors'])
directors_matrix = vectorizer_directors.transform(movies['directors_name'])
companies_matrix = vectorizer_companies.transform(movies['production_companies'])
genres_matrix = vectorizer_genres.transform(movies['genres'])

# Transformar datos de futuras películas
actors_matrix_futuras = vectorizer_actors.transform(futuras['actors'])
directors_matrix_futuras = vectorizer_directors.transform(futuras['directors_name'])
companies_matrix_futuras = vectorizer_companies.transform(futuras['production_companies'])
genres_matrix_futuras = vectorizer_genres.transform(futuras['genres'])

# Concatenar todas las características
X = hstack([actors_matrix, directors_matrix, companies_matrix, genres_matrix])

# Estandarizar características numéricas
scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(X)

# Guardar los vectorizadores y el escalador
joblib.dump(vectorizer_actors, 'vectorizer_actors.pkl')
joblib.dump(vectorizer_directors, 'vectorizer_directors.pkl')
joblib.dump(vectorizer_companies, 'vectorizer_companies.pkl')
joblib.dump(vectorizer_genres, 'vectorizer_genres.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Variable objetivo
y = np.log1p(movies['revenue'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo KNeighborsRegressor con GridSearchCV
param_grid = {'n_neighbors': [5, 10, 20, 50]}
knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')
start_time = time.time()
grid_search.fit(X_train, y_train)
end_time = time.time()
print(f'Tiempo de entrenamiento (KNeighborsRegressor): {end_time - start_time} segundos')

# Mejor modelo KNeighborsRegressor
best_knn = grid_search.best_estimator_

# Guardar el mejor modelo
joblib.dump(best_knn, 'revenue_predictor_knn.pkl')

# Predecir y evaluar el modelo
y_pred_log = best_knn.predict(X_test)
y_pred = np.expm1(y_pred_log)  # Transformación inversa del logaritmo
y_test_original = np.expm1(y_test)  # Transformación inversa del logaritmo

mse = mean_squared_error(y_test_original, y_pred)
print(f'Error cuadrático medio (KNeighborsRegressor): {mse}')

# Probar con GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=200, random_state=42)
start_time = time.time()
gbr.fit(X_train, y_train)
end_time = time.time()
print(f'Tiempo de entrenamiento (GradientBoostingRegressor): {end_time - start_time} segundos')

# Guardar el modelo GradientBoostingRegressor
joblib.dump(gbr, 'revenue_predictor_gbr.pkl')

# Predecir y evaluar el modelo
y_pred_log_gbr = gbr.predict(X_test)
y_pred_gbr = np.expm1(y_pred_log_gbr)  # Transformación inversa del logaritmo
y_test_original = np.expm1(y_test)  # Transformación inversa del logaritmo

mse_gbr = mean_squared_error(y_test_original, y_pred_gbr)
print(f'Error cuadrático medio (GradientBoostingRegressor): {mse_gbr}')

# Seleccionar las 10 películas con el revenue más alto del conjunto de prueba
top_10_indices = y_test_original.nlargest(10).index
top_10_movies = movies.loc[top_10_indices]
top_10_real_revenue = y_test_original.loc[top_10_indices]
top_10_predicted_revenue = pd.Series(y_pred_gbr, index=y_test.index).loc[top_10_indices]

# Crear un DataFrame con los resultados y formatear los valores
result_df = pd.DataFrame({
    'title': top_10_movies['title'].values,
    'real_revenue': top_10_real_revenue.apply(lambda x: f"${x:,.0f}"),
    'predicted_revenue': top_10_predicted_revenue.apply(lambda x: f"${x:,.0f}")
})

print(result_df)

# Ahora aplicamos el modelo sobre las futuras películas
# Transformar y escalar las características usando los objetos cargados
X_futuras = hstack([actors_matrix_futuras, directors_matrix_futuras, companies_matrix_futuras, genres_matrix_futuras])
X_futuras = scaler.transform(X_futuras)

# Predecir con el modelo entrenado (GradientBoostingRegressor en este caso)
y_futuras_pred_log = gbr.predict(X_futuras)
y_futuras_pred = np.expm1(y_futuras_pred_log)

futuras['predicted_revenue'] = y_futuras_pred

# Seleccionar las 10 películas con la predicción de revenue más alto
top_10_futuras = futuras.nlargest(10, 'predicted_revenue').copy()

# Formatear los valores de predicción de revenue
top_10_futuras['predicted_revenue'] = top_10_futuras['predicted_revenue'].apply(lambda x: f"${x:,.0f}")

print(top_10_futuras[['title', 'predicted_revenue']])


Tiempo de entrenamiento (KNeighborsRegressor): 12.547938585281372 segundos
Error cuadrático medio (KNeighborsRegressor): 1.942748963027762e+16
Tiempo de entrenamiento (GradientBoostingRegressor): 138.80754685401917 segundos
Error cuadrático medio (GradientBoostingRegressor): 6.876916331488677e+16
                                           title    real_revenue  \
15                             Avengers: Endgame  $2,800,000,000   
248               Jurassic World: Fallen Kingdom  $1,310,466,296   
99                                        Frozen  $1,274,219,009   
113                               Captain Marvel  $1,131,416,446   
428              Transformers: Age of Extinction  $1,104,054,072   
330             Star Wars: The Rise of Skywalker  $1,074,144,248   
149  Pirates of the Caribbean: On Stranger Tides  $1,045,713,802   
538                              Despicable Me 3  $1,031,552,585   
101                                     Zootopia  $1,023,784,195   
59        Harry Potter